In [ ]:
import os
import sys
import subprocess
import json
import textwrap
import pathlib
from pathlib import Path
import shutil

REPO_URL = "https://github.com/willloe/MSML640_Group10.git"
REPO_DIR = "/content/MSML640_Group10"
BRANCH = "feature/LoRA"

if not pathlib.Path(REPO_DIR).exists():
    subprocess.run(["git", "clone", REPO_URL, REPO_DIR], check=True)
else:
    print("Repo already present; pulling latest...")
    subprocess.run(["git", "-C", REPO_DIR, "pull", "--ff-only"], check=True)

subprocess.run(["git", "-C", REPO_DIR, "fetch", "origin", BRANCH], check=True)
subprocess.run(["git", "-C", REPO_DIR, "checkout", BRANCH], check=True)
subprocess.run(["git", "-C", REPO_DIR, "pull", "--ff-only"], check=True)

SRC = f"{REPO_DIR}/packages/diffusion/src"
if SRC not in sys.path:
    sys.path.append(SRC)

import torch
print("CUDA available:", torch.cuda.is_available())
print("CUDA device name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")

print("\nSmoke_sdxl_load")
subprocess.run(["python", f"{REPO_DIR}/scripts/smoke_sdxl_load.py"], check=False)

print("\nSmoke_synthetic")
subprocess.run(["python", f"{REPO_DIR}/scripts/smoke_synthetic.py"], check=False)

print("\nSmoke_infer (base)")
subprocess.run(["python", f"{REPO_DIR}/scripts/smoke_infer.py"], check=False)

print("\nSmoke_infer (controlnet edge, dpmpp)")
subprocess.run([
    "python", f"{REPO_DIR}/scripts/smoke_infer.py",
    "--use_controlnet", "1",
    "--control_from", "edge",
    "--scheduler", "dpmpp",
    "--steps", "14",
    "--guidance", "6.0",
    "--width", "1024",
    "--height", "768",
    "--seed", "1234",
], check=False)

print("\nSmoke_upscale_inpaint")
subprocess.run(["python", f"{REPO_DIR}/scripts/smoke_upscale_inpaint.py"], check=False)

LORA_STYLE_DIR = Path(REPO_DIR) / "data" / "lora_style"
LORA_STYLE_DIR.mkdir(parents=True, exist_ok=True)

outputs_dir = Path(REPO_DIR) / "outputs"
pngs = sorted(outputs_dir.rglob("*.png"), key=lambda p: p.stat().st_mtime) if outputs_dir.exists() else []
copied = 0
for p in reversed(pngs):
    try:
        dest = LORA_STYLE_DIR / p.name
        shutil.copy2(p, dest)
        copied += 1
        if copied >= 12:
            break
    except Exception:
        pass

print(f"\nLoRA: prepared style set with {copied} images at {LORA_STYLE_DIR}")

lora_jsonl = Path(REPO_DIR) / "outputs" / "lora" / "runs" / "exp01" / "manifests" / "captions.jsonl"
lora_jsonl.parent.mkdir(parents=True, exist_ok=True)

print("\nLoRA: prepare_lora_dataset")
subprocess.run([
    "python", f"{REPO_DIR}/scripts/prepare_lora_dataset.py",
    "--images_dir", str(LORA_STYLE_DIR),
    "--out_jsonl", str(lora_jsonl),
    "--fallback_caption", "soft abstract gradient, slide-safe, minimal clutter"
], check=False)

print("\nLoRA: train_lora (short run)")
subprocess.run([
    "python", f"{REPO_DIR}/scripts/train_lora.py",
    "--images_dir", str(LORA_STYLE_DIR),
    "--output_dir", f"{REPO_DIR}/outputs/lora/runs/exp01",
    "--resolution", "768",
    "--rank", "8",
    "--batch_size", "1",
    "--gradient_accumulation_steps", "1",
    "--max_train_steps", "20"
], check=False)

final_lora_dir = Path(REPO_DIR) / "outputs" / "lora" / "runs" / "exp01" / "final_lora"
ab_out = Path(REPO_DIR) / "outputs" / "lora_ab"
ab_out.mkdir(parents=True, exist_ok=True)

print("\nLoRA: smoke_lora_ab (A/B compare)")
subprocess.run([
    "python", f"{REPO_DIR}/scripts/smoke_lora_ab.py",
    "--lora_dir", str(final_lora_dir),
    "--out_dir", str(ab_out),
    "--seed", "777",
    "--width", "768",
    "--height", "768",
    "--steps", "20",
    "--guidance", "5.5",
    "--control_mode", "safe"
], check=False)

print("\nCompleted. Check /content/MSML640_Group10/outputs for images, including LoRA A/B outputs.")